---
title       : "Assignment: pharmacy entry"
subtitle    : "Part IV-??" 
author      : Paul Schrimpf
date        : `j using Dates; print(Dates.today())`
bibliography: "entry.bib"
link-citations: true
---

<a rel="license"
href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative
Commons License" style="border-width:0"
src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png"
/></a><br />This work is licensed under a <a rel="license"
href="http://creativecommons.org/licenses/by-sa/4.0/">Creative
Commons Attribution-ShareAlike 4.0 International License</a>.

### About this document {-}

This document was created using Weave.jl. The code is available in
[on github](https://github.com/ECON567/PharmacyEntry). The same
document generates both static webpages and associated jupyter
notebooks. 

$$
\def\indep{\perp\!\!\!\perp}
\def\Er{\mathrm{E}}
\def\R{\mathbb{R}}
\def\En{{\mathbb{E}_n}}
\def\Pr{\mathrm{P}}
\newcommand{\norm}[1]{\left\Vert {#1} \right\Vert}
\newcommand{\abs}[1]{\left\vert {#1} \right\vert}
\DeclareMathOperator*{\argmax}{arg\,max}
\DeclareMathOperator*{\argmin}{arg\,min}
\def\inprob{\,{\buildrel p \over \rightarrow}\,} 
\def\indist{\,{\buildrel d \over \rightarrow}\,} 
$$

In [1]:
using Pkg 
Pkg.activate("..") 
Pkg.resolve()

using Revise
if (!("../src" ∈ LOAD_PATH))
  push!(LOAD_PATH, "../src") 
end
using PharmacyEntry

 Resolving package versions...


HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/2fdf54a55c85c93975004f0c2902566112dc6902-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/2fdf54a55c85c93975004f0c2902566112dc6902-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

  Updating `~/565/assignments/PharmacyEntry/Project.toml`
 [no changes]
  Updating `~/565/assignments/PharmacyEntry/Manifest.toml`
 [no changes]


# Part IV - Model

As in @br1991, we will assume that the profits per pharmacy in
market $m$ with $N$ pharmacies is 

$$
\begin{align*}
    \pi_{m,N} = s_m \underbrace{(\alpha_1 + x_m\beta + \sum_{n=2}^N
    \alpha_n)}_{\text{variable profits}} - \underbrace{\left(\gamma_1 + \delta
    w_m + \sum_{n=2}^N \gamma_n \right)}_{\text{fixed costs}} +
    \epsilon_m 
\end{align*}
$$

where $s_m$ is the size of the market. To simplify, I am omitting the
$\lambda$ and the other size shifting variables from the model. You may add
these if you wish. 

Let $\theta = (\alpha, \beta, \gamma)$ denote the model parameters.
If we assume $\epsilon_m$ has cdf $F_\epsilon()$ (conditional on $s$,
$x$, and $w$), then the likelihood of observing $N_m$ pharmacies in
market $m$ is

$$
\begin{align*}
   P(N = N_m | s_m, x_m, w_m; \theta) = & P(\pi_{m,N} \geq 0 \;\&\;
   \pi_{m,N+1} < 0) \\
   = & P\left(-\left[s_m (\alpha_1 + x_m\beta - \sum_{n=2}^{N_m}
    \alpha_n) - \left(\gamma_1 + \delta
    w_m + \sum_{n=2}^{N_m} \gamma_n \right)\right] \leq
    \epsilon_m \leq -\left[s_m (\alpha_1 + x_m\beta - \sum_{n=2}^{N_m+1}
    \alpha_n) - \left(\gamma_1 + \delta
    w_m + \sum_{n=2}^{N_m+1} \gamma_n \right)\right] \right) \\
   = & F_\epsilon\left(-\left[s_m (\alpha_1 + x_m\beta - \sum_{n=2}^{N_m+1}
    \alpha_n) - \left(\gamma_1 + \delta
    w_m + \sum_{n=2}^{N_m+1} \gamma_n \right)\right]\right) -
    F_\epsilon\left( -\left[s_m (\alpha_1 + x_m\beta - \sum_{n=2}^{N_m}
    \alpha_n) - \left(\gamma_1 + \delta
    w_m + \sum_{n=2}^{N_m} \gamma_n \right)\right] \right)
\end{align*}
$$

The loglikelihood is then

$$
\mathcal{L}(\theta) = \sum_{m=1}^M \log P(N = N_m | s_m, x_m, w_m;
\theta),
$$

and $\theta$ can be estimated by maximizing,

$$
\hat{\theta} = \argmax_\theta \mathcal{L}(\theta).
$$

### Problem 1: loglikelihood

Write a function to compute the loglikelihood. You may do this however
you want, but I suggest using the following skeleton code.

In [2]:
using Distributions

"""
         brentrymodel(data::AbstractDataFrame,
                      n::Symbol,
                      s::Symbol,
                      x::Array{Symbol,1},
                      w::Array{Symbol,1};
                      Fϵ)

Create loglikelihood for Bresnehan & Reiss style entry model

Inputs:
- `data` DataFrame 
- `n` name of number of firm variable in data
- `s` name of market size variable in data
- `x` array of names of variable profit shifters
- `w` array of names of fixed cost shifters 
- `Fϵ` cdf of ϵ, optional, defaults to standard normal cdf

The same variables may be included in both `x` and `w`.


"""
function brentrymodel(data::AbstractDataFrame,
                      n::Symbol,
                      s::Symbol,
                      x::Array{Symbol,1},
                      w::Array{Symbol,1};
                      Fϵ = x->cdf(Normal(),x))
  # skip observations with missings
  vars = [n, s, x..., w...]
  inc = completecases(data[vars])

  N = disallowmissing(data[n][inc])
  S = disallowmissing(data[s][inc])
  X = disallowmissing(convert(Matrix, data[x][inc,:]))
  W = disallowmissing(convert(Matrix, data[w][inc,:]))
  Nmax = maximum(N)
  function packparam(α,β,γ,δ)
    θ = (α,β,γ,δ)
  end
  function unpackparam(θ)
    α = θ[1:Nmax]
    β = θ[(Nmax+1):(Nmax+size(X,2))]
    γ = θ[(Nmax+size(X,2)+1):(Nmax+size(X,2)+Nmax)]
    δ = θ[(Nmax+size(X,2)+Nmax+1):end]
    (α,β,γ,δ)
  end
  function loglike(θ)
    (α,β,γ,δ) = unpackparam(θ)
    error("You must write the body of this function")    
  end
  
  return(loglike=loglike, unpack=unpackparam, pack=packparam)
end

UndefVarError: UndefVarError: AbstractDataFrame not defined

### Problem 2: estimate on simulated data 

It is good practice to test any estimation method on simulated
data. The function `brentrysim` in `PharmacyEntry/src/entrymodel.jl`
simulates this model. Use it to test your likelihood. Here is some
code to simulate. You may need to adjust the parameters to get a
decent distribution of number of firms (i.e. not all 0 or 5).

In [3]:
# Simulating data
using DataFrames
import CSV
df = CSV.read("cleandata.csv")

# parameters for simulation
α = [-0.13, -0.29, -0.19, -0.2, -0.1] # loosely taken from Bresnehan & Reiss
                                      # Table 4, note that my
                                      # notation has α[2:n] equal to
                                      # their -α[2:n]
γ = [0.5,   1.,  1.,  1., 1.]
# you may have to adjust the parameters to get a reasonable distribution of
# number of pharmacies across markets
svar = Symbol("Population, 2016")
α = α./mean(df[svar])  # scale to fit this data better
β = [1., 10., 2.]
xvars = [Symbol("Employment rate"),
         Symbol("Average total income in 2015 among recipients (\$)"),
         Symbol("65 years and over")]
for j in 1:length(β)
  β[j] = β[j] / mean(df[xvars[j]].*df[svar])
end
δ = [1., 1.]
wvars = [Symbol("Population density per square kilometre"),
         Symbol("Land area in square kilometres")]
for j in 1:length(δ)
  δ[j] = δ[j] / maximum(df[wvars[j]])
end

df[:nsim] = brentrysim(df, svar, xvars, wvars, α,β,γ,δ)
println("Distribution of number of firms")
for i in 0:length(α)
  println("$(mean(df[:nsim].==i))") 
end

Distribution of number of firms
0.2468354430379747
0.2911392405063291
0.15822784810126583
0.08227848101265822
0.05063291139240506
0.17088607594936708


To estimate from the simulated data, you could do the following.

In [4]:
using Optim

(loglike, unpack, pack) = brentrymodel(df, :nsim, svar, xvars, wvars)
θ0 = pack(α,β,γ,δ)
res = optimize(loglike, zeros(size(θ0)), BFGS(), autodiff=:forward)
θhat = res.minimizer
(αhat, βhat, γhat, δhat) = unpack(θhat)

UndefVarError: UndefVarError: brentrymodel not defined

Ideally, you would do this many times, and verify that on average the
estimates are close to the true parameters. 


TO BE CONTINUED